In [7]:
import time
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os

# Configurações gerais
plt.rcParams.update({'figure.max_open_warning': 0})

# Diretório para salvar as figuras
output_dir = 'figuras_var'
os.makedirs(output_dir, exist_ok=True)

# Medição do tempo total do script
total_start_time = time.time()

# Medição do tempo de carregamento dos dados
start_time = time.time()

# Carregar os arquivos .mat fornecidos
file_path_temp_coefs = "temp_coefs.mat"
file_path_spatial_modes = "spatial_modes.mat"
file_path_mean_flow = "mean_flow.mat"
file_path_parameters = "parameters.mat"

# Carregar os coeficientes temporais
mat_data_temp = sio.loadmat(file_path_temp_coefs)
ts = mat_data_temp['ts'].flatten()  # Vetor de tempo
coefs = mat_data_temp['coefs']      # Matriz de coeficientes complexos

# Carregar os modos espaciais
mat_data_spatial = sio.loadmat(file_path_spatial_modes)
phi = mat_data_spatial['phi']       # Matriz de modos espaciais

# Carregar o vetor y e a média Xavg
mat_data_mean_flow = sio.loadmat(file_path_mean_flow)
y_positions = mat_data_mean_flow['y'].flatten()  # Vetor de posições espaciais y
Xavg = mat_data_mean_flow['Xavg'].flatten()      # Média do fluxo

# Carregar os modos usados no POD
mat_data_parameters = sio.loadmat(file_path_parameters)
nmodos_pod = mat_data_parameters['nmodes']       # Modos usados no POD

# Tempo de carregamento dos dados
print(f"Tempo de carregamento dos dados: {time.time() - start_time:.2f} segundos")

# Definir o número de coeficientes a serem utilizados
num_coefs = 10
print(f"Número total de modos utilizados: {num_coefs}")

# Separar os coeficientes em parte real e imaginária
real_coefs = np.real(coefs[:num_coefs, :])
imag_coefs = np.imag(coefs[:num_coefs, :])

# Concatenar as partes real e imaginária
coefs_combined = np.vstack([real_coefs, imag_coefs])  # Dimensão (num_coefs*2, N)

# Transpor para ter a forma (N, num_coefs*2)
coefs_combined = coefs_combined.T

# Medição do tempo de pré-processamento
start_time = time.time()

# Dividir os dados em treinamento e validação
first_interval_end = 4801  # Posição correspondente ao tempo 1300
train_data = coefs_combined[:first_interval_end:2]  # Subamostragem para taxa de 0.5
val_data = coefs_combined[first_interval_end:]

print(f"Tempo de pré-processamento: {time.time() - start_time:.2f} segundos")

# Medição do tempo de treinamento do modelo VAR
start_time = time.time()

# Ajustar o modelo VAR nos dados de treinamento
model_var = VAR(train_data)
# Selecionar automaticamente a ordem do modelo (número de defasagens) usando o critério AIC
results_aic = model_var.select_order(maxlags=15)
selected_lag = results_aic.aic
print(f"Ordem do modelo VAR selecionada (AIC): {selected_lag}")
model_fitted = model_var.fit(selected_lag)

print(f"Tempo de treinamento do modelo VAR: {time.time() - start_time:.2f} segundos")

# Fazer previsões no conjunto de validação
lag_order = model_fitted.k_ar
print(f"Ordem do modelo ajustado: {lag_order}")

# Preparar os dados para a previsão
input_data = np.concatenate([train_data[-lag_order:], val_data[:lag_order]])

# Número de previsões a serem feitas
num_predictions = len(val_data) - lag_order

# Medição do tempo de previsão
start_time = time.time()

# Fazer previsões iterativamente
forecast_results = model_fitted.forecast(y=input_data, steps=num_predictions)

print(f"Tempo de previsão: {time.time() - start_time:.2f} segundos")

# Obter as previsões correspondentes
predictions = forecast_results

# Obter os valores reais correspondentes
actual_values = val_data[lag_order:]

# Garantir que as dimensões correspondam
min_length = min(predictions.shape[0], actual_values.shape[0])
predictions = predictions[:min_length]
actual_values = actual_values[:min_length]

# Separar as partes real e imaginária das previsões e valores reais
pred_real = predictions[:, :num_coefs]
pred_imag = predictions[:, num_coefs:]
actual_real = actual_values[:, :num_coefs]
actual_imag = actual_values[:, num_coefs:]

# Reconstruir os coeficientes complexos preditos e reais
predicted_coefs = pred_real + 1j * pred_imag
actual_coefs = actual_real + 1j * actual_imag

# Reconstruir X a partir dos coeficientes preditos e dos modos espaciais
phi_reduced = phi[:, :num_coefs]  # Dimensão (387, num_coefs)
Xavg = Xavg.flatten()  # Dimensão (387,)

X_rec_list = []
X_actual_list = []

for i in range(predicted_coefs.shape[0]):
    # Reconstrução predita
    X_rec = Xavg + phi_reduced @ predicted_coefs[i]
    X_rec_list.append(X_rec)
    # Reconstrução real
    X_actual = Xavg + phi_reduced @ actual_coefs[i]
    X_actual_list.append(X_actual)

# Converter as listas em arrays e transpor
X_rec_array = np.array(X_rec_list).T        # Forma (387, num_samples)
X_actual_array = np.array(X_actual_list).T  # Forma (387, num_samples)

# Número de pontos em y
ny = y_positions.shape[0]  # Deve ser 129

# Plotar a comparação entre os perfis originais e reconstruídos de |u|
step = 10  # Número de subplots (ajuste conforme necessário)

plt.figure(figsize=(16, 12))
plt.clf()
for i in range(step):
    plt.subplot(4, 5, i + 1)
    # Índice do tempo
    idx = i * (X_rec_array.shape[1] // step)
    plt.plot(np.abs(X_actual_array[:ny, idx]), y_positions, label='Original')
    plt.plot(np.abs(X_rec_array[:ny, idx]), y_positions, label='Reconstruído', linestyle='--')
    plt.xlabel('|u|')
    plt.ylabel('y')
    plt.title(f'Amostra {idx}')
    plt.grid(True)
    if i == 0:
        plt.legend()
plt.suptitle('Comparação de |u| Original e Reconstruído (Validação) - VAR')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(os.path.join(output_dir, 'comparacao_u_original_reconstruido_var.png'), dpi=400)
plt.close()

# Calcular as métricas de erro separadamente para as partes real e imaginária
mse_real = mean_squared_error(actual_real.flatten(), pred_real.flatten())
mae_real = mean_absolute_error(actual_real.flatten(), pred_real.flatten())
r2_real = r2_score(actual_real.flatten(), pred_real.flatten())

mse_imag = mean_squared_error(actual_imag.flatten(), pred_imag.flatten())
mae_imag = mean_absolute_error(actual_imag.flatten(), pred_imag.flatten())
r2_imag = r2_score(actual_imag.flatten(), pred_imag.flatten())

print(f"Métricas para a parte real (VAR):")
print(f"MSE: {mse_real:.6f}")
print(f"MAE: {mae_real:.6f}")
print(f"R²: {r2_real:.6f}")

print(f"Métricas para a parte imaginária (VAR):")
print(f"MSE: {mse_imag:.6f}")
print(f"MAE: {mae_imag:.6f}")
print(f"R²: {r2_imag:.6f}")

# Função para calcular NMSE por coeficiente
def calculate_normalized_mse(actual, predicted):
    num_coefs = actual.shape[1]
    nmse = np.zeros(num_coefs)
    for c in range(num_coefs):
        mse = mean_squared_error(actual[:, c], predicted[:, c])
        variance = np.var(actual[:, c])
        # Evitar divisão por zero
        if variance != 0:
            nmse[c] = mse / variance
        else:
            nmse[c] = np.nan  # ou trate conforme apropriado
    return nmse

# Calcular NMSE para a parte real
nmse_real = calculate_normalized_mse(actual_real, pred_real)

# Calcular NMSE para a parte imaginária
nmse_imag = calculate_normalized_mse(actual_imag, pred_imag)

# Calcular NMSE para o módulo dos coeficientes
actual_modulus = np.sqrt(actual_real**2 + actual_imag**2)
predicted_modulus = np.sqrt(pred_real**2 + pred_imag**2)
nmse_modulus = calculate_normalized_mse(actual_modulus, predicted_modulus)

# Calcular a média do NMSE sobre todos os coeficientes
average_nmse_real = np.nanmean(nmse_real)
average_nmse_imag = np.nanmean(nmse_imag)
average_nmse_modulus = np.nanmean(nmse_modulus)

print(f"Média do NMSE para a parte real (VAR): {average_nmse_real:.6f}")
print(f"Média do NMSE para a parte imaginária (VAR): {average_nmse_imag:.6f}")
print(f"Média do NMSE para o módulo dos coeficientes (VAR): {average_nmse_modulus:.6f}")

# Plotar NMSE do Módulo por coeficiente
coefficients = np.arange(1, num_coefs + 1)

plt.figure(figsize=(12, 6))
plt.bar(coefficients, nmse_modulus, width=0.6, label='Módulo (VAR)', color='green')
plt.xlabel('Índice do Coeficiente')
plt.ylabel('NMSE')
plt.title('NMSE por Coeficiente (Módulo) - VAR')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_dir, 'nmse_por_coeficiente_modulo_var.png'), dpi=400)
plt.close()

# Plotar NMSE para a parte real, imaginária e módulo dos coeficientes
plt.figure(figsize=(14, 6))

# NMSE da Parte Real
plt.subplot(1, 3, 1)
plt.bar(coefficients, nmse_real, width=0.6, color='blue')
plt.xlabel('Índice do Coeficiente')
plt.ylabel('NMSE')
plt.title('NMSE por Coeficiente - Parte Real (VAR)')
plt.grid(True)

# NMSE da Parte Imaginária
plt.subplot(1, 3, 2)
plt.bar(coefficients, nmse_imag, width=0.6, color='green')
plt.xlabel('Índice do Coeficiente')
plt.ylabel('NMSE')
plt.title('NMSE por Coeficiente - Parte Imaginária (VAR)')
plt.grid(True)

# NMSE do Módulo
plt.subplot(1, 3, 3)
plt.bar(coefficients, nmse_modulus, width=0.6, color='orange')
plt.xlabel('Índice do Coeficiente')
plt.ylabel('NMSE')
plt.title('NMSE por Coeficiente - Módulo (VAR)')
plt.grid(True)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'nmse_por_coeficiente_todas_var.png'), dpi=400)
plt.close()

# Plotar os coeficientes reais e preditos para alguns coeficientes selecionados
num_plots = 4  # Número de coeficientes a serem plotados
random_indices = np.random.choice(num_coefs, num_plots, replace=False)

plt.figure(figsize=(15, 10))
for idx, coef_index in enumerate(random_indices):
    plt.subplot(2, 2, idx + 1)
    plt.plot(actual_real[:, coef_index], label=f'Parte Real - Coeficiente {coef_index+1} (Real)')
    plt.plot(pred_real[:, coef_index], label=f'Parte Real - Coeficiente {coef_index+1} (Predito)', linestyle='--')
    plt.title(f'Comparação da Parte Real do Coeficiente {coef_index+1} (Validação) - VAR')
    plt.xlabel('Amostra de Tempo')
    plt.ylabel('Valor')
    plt.legend()
    plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'comparacao_parte_real_coeficientes_var.png'), dpi=400)
plt.close()

plt.figure(figsize=(15, 10))
for idx, coef_index in enumerate(random_indices):
    plt.subplot(2, 2, idx + 1)
    plt.plot(actual_imag[:, coef_index], label=f'Parte Imaginária - Coeficiente {coef_index+1} (Real)')
    plt.plot(pred_imag[:, coef_index], label=f'Parte Imaginária - Coeficiente {coef_index+1} (Predito)', linestyle='--')
    plt.title(f'Comparação da Parte Imaginária do Coeficiente {coef_index+1} (Validação) - VAR')
    plt.xlabel('Amostra de Tempo')
    plt.ylabel('Valor')
    plt.legend()
    plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'comparacao_parte_imaginaria_coeficientes_var.png'), dpi=400)
plt.close()

# Plotar evolução do NMSE ao longo das amostras de tempo
nmse_time = np.mean((actual_values - predictions)**2, axis=1) / np.var(actual_values, axis=0).mean()

plt.figure(figsize=(12, 6))
plt.plot(nmse_time, label='NMSE ao Longo do Tempo')
plt.xlabel('Amostra de Tempo')
plt.ylabel('NMSE')
plt.title('Evolução do NMSE ao Longo das Amostras de Tempo - VAR')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_dir, 'nmse_tempo_var.png'), dpi=400)
plt.close()

# Adicionar a plotagem da evolução do NMSE ao longo do tempo
nmse_time = np.mean((actual_values - predictions)**2, axis=1) / np.var(actual_values, axis=0).mean()

plt.figure(figsize=(12, 6))
plt.plot(nmse_time, label='NMSE ao Longo do Tempo')
plt.xlabel('Amostra de Tempo')
plt.ylabel('NMSE')
plt.title('Evolução do NMSE ao Longo das Amostras de Tempo - VAR')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_dir, 'nmse_tempo_var.png'), dpi=400)
plt.close()

# Plotar a comparação dos perfis de velocidade 'v' e 'w' originais e reconstruídos

# Exemplo para 'v'
plt.figure(figsize=(16, 12))
plt.clf()
for i in range(step):
    plt.subplot(4, 5, i + 1)
    idx = i * (X_rec_array.shape[1] // step)
    plt.plot(np.abs(X_actual_array[ny:2*ny, idx]), y_positions, label='Original')
    plt.plot(np.abs(X_rec_array[ny:2*ny, idx]), y_positions, label='Reconstruído', linestyle='--')
    plt.xlabel('|v|')
    plt.ylabel('y')
    plt.title(f'Amostra {idx}')
    plt.grid(True)
    if i == 0:
        plt.legend()
plt.suptitle('Comparação de |v| Original e Reconstruído (Validação) - VAR')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(os.path.join(output_dir, 'comparacao_v_original_reconstruido_var.png'), dpi=400)
plt.close()

# Exemplo para 'w'
plt.figure(figsize=(16, 12))
plt.clf()
for i in range(step):
    plt.subplot(4, 5, i + 1)
    idx = i * (X_rec_array.shape[1] // step)
    plt.plot(np.abs(X_actual_array[2*ny:, idx]), y_positions, label='Original')
    plt.plot(np.abs(X_rec_array[2*ny:, idx]), y_positions, label='Reconstruído', linestyle='--')
    plt.xlabel('|w|')
    plt.ylabel('y')
    plt.title(f'Amostra {idx}')
    plt.grid(True)
    if i == 0:
        plt.legend()
plt.suptitle('Comparação de |w| Original e Reconstruído (Validação) - VAR')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(os.path.join(output_dir, 'comparacao_w_original_reconstruido_var.png'), dpi=400)
plt.close()


# Exibir o tempo total de execução do script
total_end_time = time.time()
print(f"Tempo total de execução do script: {total_end_time - total_start_time:.2f} segundos")


Tempo de carregamento dos dados: 0.14 segundos
Número total de modos utilizados: 10
Tempo de pré-processamento: 0.00 segundos
Ordem do modelo VAR selecionada (AIC): 9
Tempo de treinamento do modelo VAR: 0.96 segundos
Ordem do modelo ajustado: 9
Tempo de previsão: 0.26 segundos
Métricas para a parte real (VAR):
MSE: 0.000002
MAE: 0.000871
R²: 0.000036
Métricas para a parte imaginária (VAR):
MSE: 0.000002
MAE: 0.000872
R²: 0.000061
Média do NMSE para a parte real (VAR): 1.000108
Média do NMSE para a parte imaginária (VAR): 0.999750
Média do NMSE para o módulo dos coeficientes (VAR): 4.600995
Tempo total de execução do script: 76.10 segundos
